# Assignment: ETL/Clinical Data Warehouse 

Submission to Prof. Vianney Jouhet

Submission by Arulmani Thiyagarajan

## Highlights: 
* Steps followed:understanded in the nutshell
    1. ETL extract
    2. ETL transform
    3. ETL transform
    4. ETL load
* i2b2 is open source software mainly focuses on the translational research groups, has ability to integrate the clinical, biological and genomic data.
* Important highlight is that the model is in denormalized form
* 5 main tables are considered patient, venue, provider, observation, concept and modifier

In [ ]:
# Step:1 - Importing required libraries

import csv # this library for importing dataset
import psycopg2 # this library is for PostGregSQL adapter for python language

In [ ]:
# Step:2 - Creating the class patient with columns patient number, gender and birth date along with print option
class Patient:
    def __init__ (self,patient_num,gender,birthdate):
        self.patient_num = patient_num
        self.gender = gender
        self.birth_date = birthdate

    def printPatient(self):
        print(" *******************")
        print("Patient")
        print(self.patient_num)
        print(self.gender)
        print(self.birth_date)
        print(" *******************")

In [ ]:
# Creating the class visit with columns patient number, encounter number, start date and end date along with print option

class Visit:
    def __init__ (self, encounter_num, patient_num, start_date, end_date):
        self.encounter_num = encounter_num
        self.patient_num = patient_num
        self.start_date = start_date
        self.end_date = end_date
        
    def printVisit(self):
        print(" *******************")
        print("Visit")
        print(self.encounter_num)
        print(self.patient_num)
        print(self.start_date)
        print(self.end_date)
        print(" *******************")

In [ ]:
# Creating Observation class with encounter number, patient number, concept_cd, start date and end date
class observation:
    def __init__ (self, encounter_num, patient_num, concept_cd, start_date, end_date):
        self.encounter_num = encounter_num
        self.patient_num = patient_num
        self.concept_cd = concept_cd
        self.provider_id = 1 # keeping the provider id as one for the all the patient
        self.start_date = start_date
        self.end_date = end_date
       

    def printobservation(self):
        print(" *******************")
        print("Observation Fact")
        print(self.encounter_num)
        print(self.patient_num)
        print(self.concept_cd)
        print(self.provider_id)
        print(self.start_date)
        print(self.end_date)
        print(" *******************")

In [ ]:
# Step:3 - Defining the execute function for connecting with PostgreSQL- i2b2 database through the local host 
def execute(command):
    conn = None
    try:
        # connecting to the PostgreSQL server
        conn = psycopg2.connect(host="localhost", database="i2b2", user="i2b2", password="i2b2")
        cur = conn.cursor()
        # creating table one by one
        cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # committing the changes that we made
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error: # if not making an connection, making it to print error statement
        print(error)
    finally:
        if conn is not None: # if connection is achieved, it tells to stop the open connection once the commited changes made
            conn.close()

In [ ]:
# Step:4 - Removing the dummy data from the patient dimension, visit dimension and observationfact variable in the database

stmt = '''
delete from i2b2demodata.patient_dimension;
delete from i2b2demodata.visit_dimension;
delete from i2b2demodata.observation_fact;
'''

execute(stmt) # using the execute function that we created in the previous step, the only dummy values data are deleted from the database keeping the column names (structure) intact

In [ ]:
# Step:5 - Defining the loadPatient function to insert the given data into the columns that we need to get loaded in the database
def loadPatient(patient):
    command = """
        INSERT INTO i2b2demodata.patient_dimension
            (patient_num, sex_cd, birth_date)
            VALUES(
                    {patient_num},
                    {gender},
                    DATE '{birth_date}'
            )
    """.format(patient_num = patient.patient_num, gender = patient.gender, birth_date = patient.birth_date)
    execute(command)

In [ ]:
# Defining the loadvisit function to insert the given data into the columns that we need to get loaded in the database
def loadVisit(visit): 
    command = """
         INSERT INTO i2b2demodata.visit_dimension
            (encounter_num, patient_num, start_date, end_date)
            VALUES(
                    {encounter_num},
                    {patient_num},
                    DATE '{start_date}',
                    DATE '{end_date}'
            )
    """.format(encounter_num = visit.encounter_num, patient_num = visit.patient_num,  
               start_date = visit.start_date, end_date = visit.end_date)
    execute(command)

In [ ]:
# Defining the loadobservation function to insert the given data into the columns that we need to get loaded in the database
def loadobservation(observation):
    command = """
        INSERT INTO i2b2demodata.observation_fact
            (encounter_num, patient_num, concept_cd, provider_id, start_date, end_date)
            VALUES(
                    {encounterNum},
                    {patientNum},
                    TEXT '{conceptCd}',
                    {providerId},
                    DATE '{start_date}',
                    DATE '{end_date}'                   
            )
    """.format(encounterNum = observation.encounter_num, patientNum = observation.patient_num, 
               conceptCd = observation.concept_cd, providerId = observation.provider_id, 
               start_date = observation.start_date, end_date = observation.end_date)
    execute(command)

In [ ]:
# Step:6 - Defining the read file function with adjustment or formatting as per our dataset

def readFile(file):
    loaded = []
    with open(file) as csvFile: 
        reader = csv.reader(csvFile, delimiter = '|') # reading the file specifying the values are delimited with '|' symbol and csv file format(executing through csv library created in first step)
        l = 0
        for row in reader:
            if l == 0:
                print(f'columns ==>  {", ".join(row)}')
            else:
                if row[0] not in loaded:
                    pat = Patient(row[0],row[1],row[2])
                    loaded.append(row[0])
                    pat.printPatient()
                    loadPatient(pat)
            
              
            l += 1
            print(l)

In [ ]:
# Step:7 - Loading our dataset into the readFile function for getting loaded into the database
file = "MPDHS-ETL/data/PMSI.txt"
readFile(file)

In [ ]:
# defining the read file for visit dimension
def readFile(file):
    loaded = []
    with open(file) as csvFile:
        reader = csv.reader(csvFile, delimiter = '|')
        l = 0
        for row in reader:
            if l == 0:
                print(f'columns ==>  {", ".join(row)}')
            else:
                if row[11] not in loaded:
                    pat = Visit(row[11], row[0], row[5], row[6])
                    loaded.append(row[11])
                    pat.printVisit()
                    loadVisit(pat)
              
            l += 1
            print(l)

In [ ]:
file = "MPDHS-ETL/data/PMSI.txt"
readFile(file)

In [ ]:
# Defining function readfile for the Observation fact

def readFile(file):
    loaded = []
    with open(file) as csvFile:
        reader = csv.reader(csvFile, delimiter = '|')
        l = 0
        for row in reader:
            if l == 0:
                print(f'columns ==>  {", ".join(row)}')
            else:
                
                    DP = observation(row[11], row[0], row[9], row[7], row[8])
                    DR = observation(row[11], row[0], row[10], row[7], row[8])
                    DAS = observation(row[11], row[0], row[12], row[7], row[8])
                
                    DP.printobservation()
                    DR.printobservation()
                    DAS.printobservation()
                      
                    loadobservation(DP)
                    loadobservation(DR)
                    loadobservation(DAS)
                      
                           
              
            l += 1
            print(l)

In [ ]:
file = "MPDHS-ETL/data/PMSI.txt"
readFile(file)
# In the output executed for the observation fact, it says duplicate key value violated unique constraint "observation_fact_pk". I do not understand this violation.